In [3]:
import os
import sys
import time
import pickle
import prince
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.cm import get_cmap
from matplotlib.font_manager import FontProperties

# 獲取當前工作目錄
current_dir = os.getcwd()
version3_path = os.path.join(current_dir, "TrafficTDApython", "Version3", "tdamapper", "core_old.py")

from sklearn.utils import resample
from sklearn.cluster import AgglomerativeClustering
from tdamapper.core_old import MapperAlgorithm
from tdamapper.cover import CubicalCover
from tdamapper.clustering import FailSafeClustering
from tdamapper.plot import MapperLayoutInteractive, MapperPlot

from models import *
from utils_v3 import *
from plots import *

try:
    myfont = FontProperties(fname=r"/System/Library/Fonts/PingFang.ttc")
    sns.set(style="whitegrid", font=myfont.get_name())
except Exception as e:
    print(e)

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

In [42]:
dataA2 = pd.read_csv("../Version3/Data/A2.csv", low_memory=False)
dataA1 = pd.read_csv("../Version3/Data/A1.csv")

In [50]:
# car data
car_0 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_0.csv"), encoding='utf-8')
car_1 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_1.csv"), encoding='utf-8')
car_2 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_2.csv"), encoding='utf-8')
car_3 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_3.csv"), encoding='utf-8')
car_21 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_21.csv"), encoding='utf-8')
car_out = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_out.csv"), encoding='utf-8')
# pass data
pass_data = pd.read_csv(os.path.join(current_dir, "Data/PassData/full.csv"), encoding='utf-8')
# full data
full_car = pd.concat([car_0, car_1, car_2, car_3, car_21, car_out], axis=0)
full_car['當事者行動狀態大類別名稱'] = 'car'
pass_data['當事者行動狀態大類別名稱'] = 'pass'
full_data = pd.concat([car_0, car_1, car_2, car_3, car_21, car_out, pass_data], axis=0)

In [60]:
select_lst = [
    '天候名稱', '光線名稱', 
    '道路類別-第1當事者-名稱', '速限-第1當事者', 
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    '當事者屬-性-別名稱', '當事者事故發生時年齡',
    '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱',
    '肇事逃逸類別名稱-是否肇逃',

    # 大類別
    '道路型態大類別名稱', '事故位置大類別名稱',
    '車道劃分設施-分向設施大類別名稱',
    '事故類型及型態大類別名稱', '當事者區分-類別-大類別名稱-車種',
    '車輛撞擊部位大類別名稱-最初', '車輛撞擊部位大類別名稱-其他',

    # 兩個欄位只有兩個觀察值不同
    '肇因研判大類別名稱-主要',
    # '肇因研判大類別名稱-個別',
]

# select data
car_0 = car_0[select_lst]
car_1 = car_1[select_lst]
car_2 = car_2[select_lst]
car_3 = car_3[select_lst]
car_21 = car_21[select_lst]
car_out = car_out[select_lst]
pass_data = pass_data[select_lst]
select_lst.append('當事者行動狀態大類別名稱')
full_data = full_data[select_lst]

In [8]:
def get_train_test_data(input_data):
    input_data['y'] = input_data['死亡'].apply(lambda x: 1 if x >= 1 else 0)
    
    new_input_data = input_data.drop(columns=['受傷', '死亡'], inplace=False)
    
    X = new_input_data.drop(columns=['y'])
    y = new_input_data['y']

    return X, y

Xfull, yfull = get_train_test_data(dummy_data)
Xfull = Xfull.astype(float)

In [ ]:
full_svc_matrix, full_svc_score, full_svc_cm, full_svc_y_resampled_test, full_svc_decision_scores = logistic_cm_gridsearch(Xfull, yfull, threshold=5)

In [30]:
y_pred = (full_svc_decision_scores >= 0.4).astype(int)

conf_matrix = confusion_matrix(full_svc_y_resampled_test, y_pred)

print(conf_matrix)
print(accuracy_score(full_svc_y_resampled_test, y_pred))

[[90 30]
 [37 83]]
0.7208333333333333
